# imports

In [3]:
import sys
sys.path.append('../')

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

import json
import numpy as np
from models.patch_to_score.dataset import PatchToScoreCrossValidationDataset
from models.patch_to_score.models.modules.transformer_encoder_mlp import TransformerEncoderMLP
from models.patch_to_score.models.modules.global_sum_pooling import GlobalSumPooling

2025-10-02 13:27:55.410611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-02 13:28:04.989513: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib
2025-10-02 13:28:04.990261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib
2025-10-02 1

# constants

In [4]:
model_results_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950'
fold = 0

# load model

In [5]:
with open(f'{model_results_dir_path}/configuration.json', 'rb') as f:
    configuration = json.load(f)
model_configuration = configuration['model']

In [6]:
from models.patch_to_score.bootstrappers.model import build_model_from_configuration

In [7]:
model = build_model_from_configuration(**model_configuration)
model.load_weights(f'{model_results_dir_path}/fold_{fold}/model.keras')

2025-10-02 13:28:24.437009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


initializing projection for skip connection: 11 -> 512


# analysis

In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 hot_encoded_value_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['hot_encoded_value_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[0][0]']         
 ingOpLambda)                                                                                     
                                                                                              

# inference

In [9]:
cross_validation_dataset = PatchToScoreCrossValidationDataset(**configuration['data'])
dataset = cross_validation_dataset.fold_datasets[fold]
batch = [*dataset.validation_set, dataset.validation_sizes, dataset.validation_num_patch]

folds amount: 5


In [8]:
predictions = model(batch)
saved_predictions = np.load(f'{model_results_dir_path}/fold_{fold}/validation/predictions.npy')

In [9]:
np.allclose(predictions, saved_predictions)

True

In [2]:
import pickle
def load_as_pickle(file_path):
    """
    Loads an object from a file using pickle.

    Parameters:
    file_path (str): The path to the file where the object is saved.

    Returns:
    any: The object loaded from the file.
    """
    with open(file_path, 'rb') as file:
        return pickle.load(file)

# all_predictions = load_as_pickle('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/21_07_with_pesto/folds_training_dicts.pkl')


In [11]:
significances_and_str_patches = load_as_pickle('/home/iscb/wolfson/omriyakir/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950/candidates/significances_and_str_patches.pkl')

In [16]:
print(significances_and_str_patches.keys())


dict_keys(['all_original_inference_predictions', 'all_original_predictions', 'all_original_K_values', 'all_mod_preds', 'all_mod_inference_predictions', 'all_mod_K_values', 'all_significants', 'all_uniprots', 'all_str_patches', 'all_labels'])


In [22]:
all_predictions_path = '/home/iscb/wolfson/omriyakir/ubinet/datasets/patch_to_score/data_for_training/21_07_with_pesto/folds_training_dicts.pkl'
proteins_path = '/home/iscb/wolfson/omriyakir/ubinet/datasets/patch_to_score/patches_dicts/21_07_with_pesto/merged_protein_objects.pkl'
all_predictions = load_as_pickle(all_predictions_path)
proteins = load_as_pickle(proteins_path)


In [4]:
data_path = '/home/iscb/wolfson/omriyakir/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950/candidates/significances_and_str_patches.pkl'
data = load_as_pickle(data_path)
print(data.keys())

dict_keys(['all_original_inference_predictions', 'all_original_predictions', 'all_original_K_values', 'all_mod_preds', 'all_mod_inference_predictions', 'all_mod_K_values', 'all_significants', 'all_uniprots', 'all_str_patches', 'all_labels'])
dict_keys(['all_original_inference_predictions', 'all_original_predictions', 'all_original_K_values', 'all_mod_preds', 'all_mod_inference_predictions', 'all_mod_K_values', 'all_significants', 'all_uniprots', 'all_str_patches', 'all_labels'])


In [2]:
import pandas as pd
path = '/home/iscb/wolfson/doririmon/home/order/bio-explore/data/esmc_layer_32_ubiquitin_classification/ensemble_test_predictions/end_epoch_6/all_folds.parquet'
df = pd.read_parquet(path)
uniprots = ['Q96PX1','O43294','Q96NL0']
df[df['uniprot'].isin(uniprots)]

,uniprot,label,sequence,type,prediction,positive_prediction
44528,O43294,0,MEDLDALLSDLETTTSHMPRSGAPKERPAEPLTPPPSYGHQPQTGS...,test,"[4.479797840118408, -2.687805652618408]",0.000771
46643,Q96NL0,0,MASRSLGGLSGIRGGGGGGGKKSLSARNAAVERRNLITVCRFSVKT...,test,"[3.267486810684204, -2.0431671142578125]",0.004914
89996,Q96PX1,0,MGALTSRQHAGVEEVDIPSNSVYRYPPKSGSYFASHFIMGGEKFDS...,test,"[-1.141250729560852, 0.9785661697387695]",0.892814
